In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd

In [2]:
model = pyo.AbstractModel()

In [3]:
room_population = pd.read_csv("SRO_input_grades.csv")

In [4]:
room_population

,Grade_ID,Population
0,PreKA,12
1,PreKB,14
2,PreKC,10
3,KindergartenA,16
4,KindergartenB,18
5,FirstA,15
6,FirstB,16
7,SecondA,32
8,ThirdA,16
9,ThirdB,17


In [5]:
room_population['Population'].sum()

238

In [6]:
model = pyo.AbstractModel()

In [7]:
model.GRADES = pyo.Set()
model.TIMESLOTS = pyo.Set()
model.ROOMS = pyo.Set()

In [8]:
model.assign = pyo.Var(model.GRADES, model.TIMESLOTS, model.ROOMS, domain = pyo.Binary)

In [9]:
model.POP = pyo.Param(model.GRADES)
model.CAP = pyo.Param(model.ROOMS)

In [10]:
@model.Objective(sense = pyo.maximize)
def profit(model):
    revenue = pyo.sum_product(model.assign)
    
    return revenue

In [11]:
@model.Constraint(model.TIMESLOTS, model.ROOMS)
def raw_material_constraint(model, t, r):
    return pyo.quicksum(model.assign[g, t, r] for g in model.GRADES) <= 1

In [12]:
@model.Constraint(model.GRADES, model.GRADES)
def equality(model, g, g_prime):
    if g == g_prime:
        return pyo.Constraint.Skip
    return (sum(model.assign[g, t, r] for t in model.TIMESLOTS for r in model.ROOMS) == 
                        sum(model.assign[g_prime, t, r] for t in model.TIMESLOTS for r in model.ROOMS))


In [13]:
# @model.Constraint(model.GRADES)
# def equality_room_constraint(model, g):
#     return sum(model.assign[g, t, r] for t in model.TIMESLOTS for r in model.ROOMS) <= (len(model.TIMESLOTS) * len(model.ROOMS)) / len(model.GRADES) 

In [14]:
@model.Constraint(model.GRADES, model.TIMESLOTS)
def room(model, g, t):
    return pyo.quicksum(model.assign[g, t, r] for r in model.ROOMS) <= 1

In [15]:
@model.Constraint(model.GRADES, model.TIMESLOTS, model.ROOMS)
def room_capacity(model, g, t, r):
    return model.assign[g, t, r] * model.POP[g] <= model.CAP[r]

In [16]:
TIMESLOTS = ['8', '9', '10', '11', '12', '13', '14', '15']

In [17]:
input_rooms = pd.read_csv("SRO_input_rooms.csv")

In [18]:
input_rooms

,Room_ID,Capacity
0,A121,15
1,A120,13
2,A123,15
3,A137,12
4,A138,15
5,B142,12
6,B144,12
7,B146,12
8,B148,12
9,C153,12


In [19]:
instanceData = {None: {
    'GRADES': {None: room_population['Grade_ID']},
    'TIMESLOTS': {None: TIMESLOTS},
    'ROOMS': {None: input_rooms['Room_ID']},
    'POP': room_population.set_index(['Grade_ID']).to_dict()['Population'],
    'CAP': input_rooms.set_index(['Room_ID']).to_dict()['Capacity']
}}

In [20]:
instance = model.create_instance(instanceData)

In [21]:
# On my computer this works, to initialize opt
path = "/home/shaunak/Software/miniconda3/envs/optimization/bin/glpsol"
opt = SolverFactory('glpk', executable=path)


# Use this on other computers
# opt = SolverFactory('glpk')


sol = opt.solve(instance)

In [22]:
sol

{'Problem': [{'Name': 'unknown', 'Lower bound': 48.0, 'Upper bound': 48.0, 'Number of objectives': 1, 'Number of constraints': 2100, 'Number of variables': 1728, 'Number of nonzeros': 43200, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.20631098747253418}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [23]:
for g in instance.GRADES:
    val = 0
    for t in instance.TIMESLOTS:
        for r in instance.ROOMS:
            val += pyo.value(instance.assign[g, t, r])
    print(val)

4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
